In [186]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [187]:
dataset = pd.read_csv("u.data.csv",sep='|',names="user_id,item_id,rating,timestamp".split(","))
dataset=dataset.drop('timestamp',axis=1)
dataset = dataset.iloc[1:,:]

datauser = pd.read_csv("u.user.csv",sep='|')
datauser = datauser.rename(columns={'id ': 'user_id',' age ':'age',' gender ':'gender',' job ':'job'})

df_item = pd.read_csv('u.item.csv', sep='|')
df_item  = df_item.drop([' video release date ',' movie title ', ' release date ',' IMDb URL '], axis=1)
df_item = df_item.rename(columns={'movie id ': 'item_id'})


In [190]:
m = np.zeros((943,1682))
for i in range(len(dataset)):
    m[int(dataset.iloc[i,0])-1,int(dataset.iloc[i,1])-1]=int(dataset.iloc[i,2])

In [191]:
dataset.item_id = dataset.item_id.astype('int16')
dataset.user_id = dataset.user_id.astype('int16')
dataset.item_id = dataset.item_id.astype('category')
dataset.user_id = dataset.user_id.astype('category')

In [192]:
datauser.user_id = datauser.user_id.astype('int16')
datauser.user_id = datauser.user_id.astype('category')
df_item.item_id = df_item.item_id.astype('int16')
df_item.item_id = df_item.item_id.astype('category')

In [193]:
dataset.head()

,user_id,item_id,rating
1,196,242,3
2,186,302,3
3,22,377,1
4,244,51,2
5,166,346,1


In [194]:
dataset.isnull().any()

user_id    False
item_id    False
rating     False
dtype: bool

In [195]:
from sklearn.model_selection import train_test_split

In [196]:
import keras
from IPython.display import SVG
from keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
from keras import regularizers

n_users, n_movies = len(dataset.user_id.unique()), len(dataset.item_id.unique())
n_latent_factors = 9
def extract_weights(name, model):
    """Extract weights from a neural network model"""
    
    # Extract weights
    weight_layer = model.get_layer(name)
    weights = weight_layer.get_weights()[0]
    return weights

from sklearn.metrics import mean_absolute_error

In [197]:
rand_list = np.sort(np.random.choice(items,100))
mae_list = []
for i in rand_list:
    movie_input = keras.layers.Input(shape=[1],name='Item')
    movie_embedding = keras.layers.Embedding(n_movies+1, n_latent_factors, name='Movie-Embedding')(movie_input)
    movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)

    user_input = keras.layers.Input(shape=[1],name='User')
    user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users+1, n_latent_factors,name='User-Embedding')(user_input))


    result = keras.layers.merge([movie_vec, user_vec], mode='dot',name='dot')


    model = keras.Model([user_input, movie_input], result)

    model.compile(optimizer='adam',loss= 'mean_squared_error')
    dataset = dataset.apply(pd.Series.replace, to_replace=i, value=944)
    model.fit([dataset.user_id,dataset.item_id],dataset.rating,epochs=25,batch_size=32,verbose=0)
    movie_weights_class = extract_weights('Movie-Embedding', model)
    user_weights_class = extract_weights('User-Embedding', model)

    movie_input = keras.layers.Input(shape=[1],name='Item')
    movie_embedding = keras.layers.Embedding(n_movies+1, n_latent_factors, name='Movie-Embedding', weights=[movie_weights_class],trainable=False)(movie_input)
    movie_vec = keras.layers.Flatten(name='FlattenMovies')(movie_embedding)

    user_input = keras.layers.Input(shape=[1],name='User')
    user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users+1, n_latent_factors,name='User-Embedding')(user_input))


    result = keras.layers.merge([movie_vec, user_vec], mode='dot',name='dot')


    model = keras.Model([user_input, movie_input], result)

    model.compile(optimizer='adam',loss= 'mean_squared_error')
    dataset = dataset.apply(pd.Series.replace, to_replace=944, value=i)
    idx = dataset.user_id==i
    idx = np.where(idx)[0]
    X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[idx,0:2],dataset.iloc[idx,-1], test_size=0.2, random_state=42)
    model.fit([X_train.user_id,X_train.item_id],y_train,epochs=1000,verbose=0)
    y_hat = model.predict([X_test.user_id, X_test.item_id])
    mae_list.append(mean_absolute_error(y_test, y_hat))
    print(mean_absolute_error(y_test, y_hat))

0.6299189840044294
1.3023208141326905
0.7718314432328747
0.551983770870027
0.8689877033233643
0.6168343871831894
0.4638632226873327
0.6781960248947143
1.0551040172576904
0.5831163371050799
0.6319037222526442
0.687795877456665
0.8114548683166504
0.6700954232897077
0.9668052593866984
0.877037763595581
0.9031744798024496
0.8344522714614868
0.7252536920400766
1.0747566006400369
0.8987705508867899
0.7616925239562988
0.6817248044190584
1.1303269522530692
0.5512738625208536
0.9700011253356934
0.8064211532473564
0.6578972339630127
0.6753005027770996
0.6091581106185913
1.0830700794855754
0.7628698062896728
0.6728868051008745
0.8899155213282659
0.7761681040128072
0.5381610070244741
0.7813098090035575
0.5190095901489258
0.7670532862345377
1.351900339126587
0.950107991695404
0.6261378186089652
0.7259295747635213
0.6881854280512384
0.9219549554481841
0.777643175125122
0.6443827697208949
0.6162920309149701
0.7483986891233004
0.5779831068856376
0.7262083530426026
0.836409330368042
1.1212906403975054


In [198]:
np.mean(mae_list)

0.7886433271893655